## 모델 테스트

In [1]:
# 보안 문제로 ipynb 파일 open시마다 실행
# from google.colab import drive
# drive.mount('/content/drive') # 마운트 이름은 자유롭게 지정 가능

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import time
# import cv2
import random

import numpy as np
import pandas as pd
import seaborn as sb
import tensorflow as tf

from tensorflow.keras.models import Sequential  # class
from tensorflow.keras.models import load_model  # model 사용
from tensorflow.keras.layers import Dense       # 전결합
from tensorflow.keras.layers import Dropout     # 특정 node를 사용안함.
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

from tensorflow.keras.callbacks import EarlyStopping   # 학습 자동 중지
from tensorflow.keras.callbacks import ModelCheckpoint # 우수한 학습 모델 파일 저장
from tensorflow.keras import regularizers 
from tensorflow.keras.utils import to_categorical   # one-hot 엔코딩
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.datasets import mnist

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split # 학습셋과 테스트셋의 분리 지원
from sklearn.model_selection import StratifiedKFold  # K겹 교차 검증

from PIL import Image

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

import platform 
# Windows, Linux, Darwin
if (platform.system() == 'Windows'):  
    rc('font', family=font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name())
    path = '.' # Local
else:    
    rc('font', family='NanumBarunGothic')  # Ubuntu 18.04 기준 한글 처리
    path = '/content/drive/My Drive/kd_ml/dnn/recommend_pinterest' # Colab

os.chdir(path) # 기본 경로 설정

plt.rcParams["font.size"] = 12         # 글자 크기
# plt.rcParams["figure.figsize"] = (10, 4) # 10:4의 그래프 비율
plt.rcParams['axes.unicode_minus'] = False  # minus 부호는 unicode 적용시 한글이 깨짐으로 설정

%matplotlib inline

# ---------------------------------------------------------------------
# gpu 사용시 런타임에서 필요한 양만큼의 GPU 메모리를 할당후 자동 증가 처리
# OS 메모리도 초기화됨.
# ---------------------------------------------------------------------
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)
    
# ---------------------------------------------------------------------

In [3]:
model = load_model('./Pinterest.h5')

In [4]:
# X = [1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0] # 데이터 49개
X = [1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] # 데이터 49개
Y= 0

data = np.array([np.array(X)]) # 입력 데이터는 ndarray 타입의 2차원이어야함, 행 방향
print(data) # comma가 없음으로 ndarray 형식임.

[[1 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [5]:
p = model.predict(data)  # 모델 사용, 2차원 데이터 전달
print(p)

[[0.49131903 0.06101102 0.04446996 0.19147088 0.04432371 0.15757765
  0.00982767]]


In [6]:
print(p*100)
print(np.max(p*100))

[[49.131905    6.1011024   4.4469957  19.147087    4.432371   15.757765
   0.98276734]]
49.131905


In [7]:
index = np.argmax(p) 
print('-> index: ', index) # 0 ~ 4
per = round(np.max(p) * 100, 3)
print('-> per: ', per) 

# 강아지, 고양이, 물고기, 자전거, 축제, 등산, 캠핑

if index == 0:
    label = '강아지'
elif index == 1:
    label = '고양이'
elif index == 2:
    label = '물고기'
elif index == 3:
    label = '자전거'
elif index == 4:
    label = '축제'
elif index == 5:
    label = '등산'
elif index == 6:
    label = '캠핑'
    
result = {}
# 종류가 5개임으로 균등한 분할은 20%임으로 20% 초과시 추천
if per >= 20:
    result = {"label": label, "per": per}
else:
    result = {"label": f'가장 인접한 추천: {label}', "per": per}

print(result)

# [[0.35821876 0.28507757 0.35670367]]: 1번만 전부 선택 했는데 확률이 너무 작은 값임.

-> index:  0
-> per:  49.132
{'label': '강아지', 'per': 49.132}


In [8]:
# 함수 기반의 처리
def recommend(X, Y):
    data = np.array([np.array(X)]) # ndarray 타입의 2차원이어야함
    p = model.predict(data)  # 모델 사용, 2차원 데이터 전달
    
    index = np.argmax(p) 
    print('-> index: ', index) # 0 ~ 4
    per = round(np.max(p) * 100, 3)
    print('-> per: ', per) 

if index == 0:
    label = '강아지'
elif index == 1:
    label = '고양이'
elif index == 2:
    label = '물고기'
elif index == 3:
    label = '자전거'
elif index == 4:
    label = '축제'
elif index == 5:
    label = '등산'
elif index == 6:
    label = '캠핑'
    
if np.max(p*100) > 15:
    print(label + ' 추천 필요') 
else:
    print('구별이 확실하지 않아 추천이 어렵습니다.')
    print(f'가장 인접한 추천: {label}')     

강아지 추천 필요


In [9]:
# 강아지, 고양이, 물고기, 자전거, 축제, 등산, 캠핑
# 정직한 선택
recommend([1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0], 0)    
recommend([0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0], 1)   
recommend([0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0], 2)  
recommend([0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0], 3) 
recommend([0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0], 4) 
# 강아지 4번, 카페 1번 선택
recommend([1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0], 0) 

-> index:  0
-> per:  48.336
-> index:  1
-> per:  53.995
-> index:  2
-> per:  82.294
-> index:  3
-> per:  84.994
-> index:  4
-> per:  53.808
-> index:  5
-> per:  22.994


In [10]:
# 참고
# 파일명을 전달받아 예측하는 함수
# 강아지, 고양이, 물고기, 자전거, 축제, 등산, 캠핑
def recommendation(fname):
    data = np.loadtxt(fname, delimiter=',', skiprows=1, dtype=np.float64)
    data = np.array([data]) # 2차원 배열로 변경, 행 방향
    
    p = model.predict(data)  # 2차원 데이터 전달하여 분류에 포함될 확률을 2차원 배열로 전달받음
    
    print(fname)
    print(p[0]) # 예측된 확률 출력
    
    index = np.argmax(p[0]) # 가장 큰 값이 있는 배열의 index return
    print('가장 큰 값이 있는 배열의 index:', index)
    if index == 0:
        label = '강아지'
    elif index == 1:
        label = '고양이'
    elif index == 2:
        label = '물고기'
    elif index == 3:
        label = '자전거'
    elif index == 4:
        label = '축제'
    elif index == 5:
        label = '등산'
    elif index == 6:
        label = '캠핑'
        
    return label

In [11]:
# 테스트
for i in range(25):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
